In [ ]:
import pandas as pd
import numpy as np
import warnings
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt
import datetime
warnings.simplefilter("ignore")

filename = 'C:\Bee Temp Data\C294_w_Env.csv'


data = pd.read_csv(filename)

input_cycle = 7 #how manys days to be used as input?
output_cycle = 14 #how manys days to be forecasted ahead?
daily_data_points = 24 #data points per day

In [ ]:
for i in range(len(data)):
    current_date = (datetime.datetime.strptime(str(data['Date'].values[i]), "%m/%d/%Y")).date()
    current_time = datetime.datetime.strptime(str(data['Time'].values[i]), "%H:%M:%S").time()
    current_datetime = datetime.datetime.combine(current_date,current_time)
    data['DateTime'][i] = datetime.datetime.strptime(str(current_datetime), "%Y-%m-%d %H:%M:%S")
    data['Date'][i] = data['DateTime'][i].date()
    data['Time'][i] = data['DateTime'][i].time()
print(data.head())

In [ ]:
def ARX_forecast():
    forecast_ARX = np.array([[],[],[],[],[],[],[],[]]).reshape(-1,8)
    id_s = 0
    for i in range(0,int(len(data)/daily_data_points)-input_cycle-output_cycle+1):
        print(i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points] #whole dataset -> input_cycle+output_cycle
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        
        exogenous_train = pd.DataFrame(d_in[0:input_cycle*daily_data_points]) #input exogenous data
        train_ext_temp = exogenous_train['Air Temp'] - np.mean(exogenous_train['Air Temp']) #removing offset/trend
        train_solar = exogenous_train['Sol Rad'] - np.mean(exogenous_train['Sol Rad'])
        train_ice = exogenous_train['P'] 
        exogenous_all_var = np.concatenate((np.linspace(0,len(exogenous_train['Air Temp'])-1,num=len(exogenous_train['Air Temp']),dtype = np.uint8).reshape(-1,1),
                                            np.array(train_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['Air Temp']),-1)
        exogenous_all_var = np.concatenate((exogenous_all_var, np.array(train_solar).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['Sol Rad']),-1)
        exogenous_all_var = np.concatenate((exogenous_all_var, np.array(train_ice).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['P']),-1)
        model = auto_arima( data_in, X = exogenous_all_var, start_p = 1, max_p = 1,
                            d = 0, max_d = 0,
                            start_q = 0, max_q = 0, information_criterion='aic')
        model.fit(data_in, X = exogenous_all_var)

        last_date_input = d_in[0:input_cycle*daily_data_points]['Date'].values[-1]
                
        forecast_whole = np.array([[],[],[],[]]).reshape(-1,4)

        forecast_data = d_in[input_cycle*daily_data_points:(input_cycle+output_cycle)*daily_data_points]
        forecast_data = forecast_data.reset_index() 
        
        for p in range(len(forecast_data)):
            forecast_data_date = datetime.datetime.strptime(str(forecast_data['Date'].values[p]),  "%Y-%m-%d").date() 
            forecast_data_time = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            forecast_data['DateTime'][p] = str(datetime.datetime.combine(forecast_data_date,forecast_data_time))
    
        #print('forecast_data',forecast_data)
        current_date = np.array([[],[],[]]).reshape(-1,3)
        for p in range(output_cycle*daily_data_points):
            current_date_forecast = (datetime.datetime.strptime(str(last_date_input), "%Y-%m-%d") + datetime.timedelta(days=int(p/daily_data_points) + 1)).date()
            current_time_forecast = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            current_datetime_forecast = datetime.datetime.combine(current_date_forecast,current_time_forecast)
            current_date = np.concatenate((current_date, np.array([str(current_date_forecast),
                                                                    str(current_time_forecast),
                                                                    str(current_datetime_forecast)]).reshape(3,-1).T))
        current_date = pd.DataFrame(current_date, columns = ['Date','Time','DateTime'])
        #print('current_data',current_date)            
        forecast_data = forecast_data.merge(current_date, how = 'outer', left_on = 'DateTime', right_on = 'DateTime')
        #print('forecast_data_merged_first',forecast_data)
        forecast_data = forecast_data[~np.isnan(forecast_data['Temp'].values)]
        forecast_data = forecast_data[~pd.isnull(forecast_data['Date_y'].values)]
        #print('forecast_data_merged',forecast_data)
        forecast_data_partial = forecast_data[['Date_y', 'Time_y', 'DateTime', 'Air Temp', 'Sol Rad', 'Temp']]
        forecast_data_partial = forecast_data_partial.rename(columns={"Date_y": "Date", "Time_y": "Time"})

        exogenous_forecast = pd.DataFrame(forecast_data_partial)
        forecast_ext_temp = exogenous_forecast['Air Temp'] - np.mean(exogenous_forecast['Air Temp'])
        forecast_solar = exogenous_forecast['Sol Rad'] - np.mean(exogenous_forecast['Sol Rad'])
        forecast_ice = [0]*len(forecast_solar)
        exogenous_all_var_forecast = np.concatenate((np.linspace(0,len(exogenous_forecast['Air Temp'])-1,num=len(exogenous_forecast['Air Temp']),dtype = np.uint8).reshape(-1,1),
                                            np.array(forecast_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Air Temp']),-1)
        exogenous_all_var_forecast = np.concatenate((exogenous_all_var_forecast, np.array(forecast_solar).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Sol Rad']),-1)
        exogenous_all_var_forecast = np.concatenate((exogenous_all_var_forecast, np.array(forecast_ice).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Sol Rad']),-1)
        
        forecast = model.predict(len(exogenous_forecast['Air Temp']), X = exogenous_all_var_forecast)
        #data_in= np.concatenate((data_in,forecast))
        id_no = [id_s]*len(forecast)
        id_s = id_s + 1
        forecast_ARX = np.concatenate((forecast_ARX, np.array([forecast_data_partial['Date'].values, 
                                                               forecast_data_partial['Time'].values,
                                                               forecast_data_partial['DateTime'].values,  
                                                               forecast_data_partial['Air Temp'].values,         
                                                               forecast_data_partial['Sol Rad'].values,
                                                               forecast_data_partial['Temp'].values,
                                                               forecast,
                                                               np.array(id_no)]).reshape(8,-1).T))
        plt.plot(range(len(forecast_data_partial['Temp'].values)), forecast_data_partial['Temp'].values, label = 'Actual')
        plt.plot(range(len(forecast_data_partial['Temp'].values)), forecast, label = 'forecast')
        plt.legend()
        plt.show()
    forecast_ARX = pd.DataFrame(forecast_ARX, columns = ['Date', 'Time', 'DateTime', 'Air Temp', 'Sol Rad',
                                                        'Temp_Actual', 'ARX', 'id_no'])
    forecast_ARX['ARX error_2'] = (forecast_ARX['Temp_Actual'] - forecast_ARX['ARX'])**2
    return forecast_ARX

In [ ]:
forecast_ARX = ARX_forecast()

In [ ]:
def Seasonal_ARX_forecast():
    forecast_sARX = np.array([[],[],[],[],[],[],[],[]]).reshape(-1,8)
    id_s = 0
    for i in range(0,int(len(data)/daily_data_points)-input_cycle-output_cycle+1):
        print(i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points] #whole dataset -> input_cycle+output_cycle
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data

        exogenous_train = pd.DataFrame(d_in[0:len(data_in)]) #input exogenous data
        train_ext_temp = exogenous_train['Air Temp'] - np.mean(exogenous_train['Air Temp']) #removing offset/trend
        train_solar = exogenous_train['Sol Rad'] - np.mean(exogenous_train['Sol Rad'])
        train_ice = exogenous_train['P'] 
        exogenous_all_var = np.concatenate((np.linspace(0,len(exogenous_train['Air Temp'])-1,num=len(exogenous_train['Air Temp']),dtype = np.uint8).reshape(-1,1),
                                            np.array(train_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['Air Temp']),-1)
        exogenous_all_var = np.concatenate((exogenous_all_var, np.array(train_solar).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['Sol Rad']),-1)
        exogenous_all_var = np.concatenate((exogenous_all_var, np.array(train_ice).reshape(-1,1)),axis = 1).reshape(len(exogenous_train['P']),-1)
        model = auto_arima( data_in, X = exogenous_all_var, start_p = 1, max_p = 1,
                            d = 0, max_d = 0,
                            start_q = 0, max_q = 0, 
                            max_P = 1, D = 0, max_D = 0, start_Q = 0, max_Q = 0,
                            m = 24, information_criterion='aic')
        model.fit(data_in, X = exogenous_all_var)
        last_date_input = d_in[0:input_cycle*daily_data_points]['Date'].values[-1]
                
        forecast_whole = np.array([[],[],[],[]]).reshape(-1,4)

        forecast_data = d_in[input_cycle*daily_data_points:(input_cycle+output_cycle)*daily_data_points]
        forecast_data = forecast_data.reset_index() 
        
        for p in range(len(forecast_data)):
            forecast_data_date = datetime.datetime.strptime(str(forecast_data['Date'].values[p]),  "%Y-%m-%d").date() 
            forecast_data_time = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            forecast_data['DateTime'][p] = str(datetime.datetime.combine(forecast_data_date,forecast_data_time))
    
        #print('forecast_data',forecast_data)
        current_date = np.array([[],[],[]]).reshape(-1,3)
        for p in range(output_cycle*daily_data_points):
            current_date_forecast = (datetime.datetime.strptime(str(last_date_input), "%Y-%m-%d") + datetime.timedelta(days=int(p/daily_data_points) + 1)).date()
            current_time_forecast = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            current_datetime_forecast = datetime.datetime.combine(current_date_forecast,current_time_forecast)
            current_date = np.concatenate((current_date, np.array([str(current_date_forecast),
                                                                    str(current_time_forecast),
                                                                    str(current_datetime_forecast)]).reshape(3,-1).T))
        current_date = pd.DataFrame(current_date, columns = ['Date','Time','DateTime'])
        #print('current_data',current_date)            
        forecast_data = forecast_data.merge(current_date, how = 'outer', left_on = 'DateTime', right_on = 'DateTime')
        #print('forecast_data_merged_first',forecast_data)
        forecast_data = forecast_data[~np.isnan(forecast_data['Temp'].values)]
        forecast_data = forecast_data[~pd.isnull(forecast_data['Date_y'].values)]
        #print('forecast_data_merged',forecast_data)
        forecast_data_partial = forecast_data[['Date_y', 'Time_y', 'DateTime', 'Air Temp', 'Sol Rad', 'Temp']]
        forecast_data_partial = forecast_data_partial.rename(columns={"Date_y": "Date", "Time_y": "Time"})
    
            #recursive forecasting
        exogenous_forecast = pd.DataFrame(forecast_data_partial)
        forecast_ext_temp = exogenous_forecast['Air Temp'] - np.mean(exogenous_forecast['Air Temp'])
        forecast_solar = exogenous_forecast['Sol Rad'] - np.mean(exogenous_forecast['Sol Rad'])
        forecast_ice = [0]*len(forecast_solar)
        exogenous_all_var_forecast = np.concatenate((np.linspace(0,len(exogenous_forecast['Air Temp'])-1,num=len(exogenous_forecast['Air Temp']),dtype = np.uint8).reshape(-1,1),
                                            np.array(forecast_ext_temp).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Air Temp']),-1)
        exogenous_all_var_forecast = np.concatenate((exogenous_all_var_forecast, np.array(forecast_solar).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Sol Rad']),-1)
        exogenous_all_var_forecast = np.concatenate((exogenous_all_var_forecast, np.array(forecast_ice).reshape(-1,1)),axis = 1).reshape(len(exogenous_forecast['Sol Rad']),-1)
        forecast = model.predict(len(exogenous_forecast['Air Temp']), X = exogenous_all_var_forecast)
        #print(forecast)
        id_no = [id_s]*len(forecast)
        id_s = id_s + 1
        forecast_sARX = np.concatenate((forecast_sARX, np.array([forecast_data_partial['Date'].values, 
                                                               forecast_data_partial['Time'].values,
                                                               forecast_data_partial['DateTime'].values,  
                                                               forecast_data_partial['Air Temp'].values,         
                                                               forecast_data_partial['Sol Rad'].values,
                                                               forecast_data_partial['Temp'].values,
                                                               forecast.values,
                                                               np.array(id_no)]).reshape(8,-1).T))
        #print(forecast_sARX)
        plt.plot(range(len(forecast_data_partial['Temp'].values)), forecast_data_partial['Temp'].values, label = 'Actual')
        plt.plot(range(len(forecast_data_partial['Temp'].values)), forecast, label = 'forecast')
        plt.legend()
        plt.show()
    forecast_sARX = pd.DataFrame(forecast_sARX, columns = ['Date', 'Time', 'DateTime', 'Air Temp', 'Sol Rad',
                                                        'Temp_Actual', 'sARX', 'id_no'])
    forecast_sARX['sARX error_2'] = (forecast_sARX['Temp_Actual'] - forecast_sARX['sARX'])**2
    return forecast_sARX

In [ ]:
forecast_sARX = Seasonal_ARX_forecast()

In [ ]:
def Holt_Winters_forecast():
    forecast_HW = np.array([[],[],[],[],[],[],[],[]]).reshape(-1,8)
    id_s = 0
    for i in range(0,int(len(data)/daily_data_points)-input_cycle-output_cycle+1):
        print(i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points]
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        model = ExponentialSmoothing(data_in,
                                     trend='add',
                                     seasonal='add',
                                     seasonal_periods = 24, 
                                     initialization_method="estimated").fit(optimized = True)
        last_date_input = d_in[0:input_cycle*daily_data_points]['Date'].values[-1]
                
        forecast_whole = np.array([[],[],[],[]]).reshape(-1,4)

        forecast_data = d_in[input_cycle*daily_data_points:(input_cycle+output_cycle)*daily_data_points]
        forecast_data = forecast_data.reset_index() 
        
        for p in range(len(forecast_data)):
            forecast_data_date = datetime.datetime.strptime(str(forecast_data['Date'].values[p]),  "%Y-%m-%d").date() 
            forecast_data_time = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            forecast_data['DateTime'][p] = str(datetime.datetime.combine(forecast_data_date,forecast_data_time))
    
        current_date = np.array([[],[],[]]).reshape(-1,3)
        for p in range(output_cycle*daily_data_points):
            current_date_forecast = (datetime.datetime.strptime(str(last_date_input), "%Y-%m-%d") + datetime.timedelta(days=int(p/daily_data_points) + 1)).date()
            current_time_forecast = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            current_datetime_forecast = datetime.datetime.combine(current_date_forecast,current_time_forecast)
            current_date = np.concatenate((current_date, np.array([str(current_date_forecast),
                                                                    str(current_time_forecast),
                                                                    str(current_datetime_forecast)]).reshape(3,-1).T))
        current_date = pd.DataFrame(current_date, columns = ['Date','Time','DateTime'])           
        forecast_data = forecast_data.merge(current_date, how = 'outer', left_on = 'DateTime', right_on = 'DateTime')
        forecast_data = forecast_data[~np.isnan(forecast_data['Temp'].values)]
        forecast_data = forecast_data[~pd.isnull(forecast_data['Date_y'].values)]
        forecast_data_partial = forecast_data[['Date_y', 'Time_y', 'DateTime', 'Air Temp', 'Sol Rad', 'Temp']]
        forecast_data_partial = forecast_data_partial.rename(columns={"Date_y": "Date", "Time_y": "Time"})
        
        forecast = model.forecast(len(forecast_data_partial))
        id_no = [id_s]*len(forecast)
        id_s = id_s + 1
        forecast_HW = np.concatenate((forecast_HW, np.array([forecast_data_partial['Date'].values, 
                                                               forecast_data_partial['Time'].values,
                                                               forecast_data_partial['DateTime'].values,  
                                                               forecast_data_partial['Air Temp'].values,         
                                                               forecast_data_partial['Sol Rad'].values,
                                                               forecast_data_partial['Temp'].values,
                                                               forecast.values,
                                                               np.array(id_no)]).reshape(8,-1).T))    
        
        plt.plot(range(len(forecast_data_partial['Temp'])), forecast_data_partial['Temp'].values, label = 'Actual')
        plt.plot(range(len(forecast_data_partial['Temp'])), forecast, label = 'forecast')
        plt.legend()
        plt.show()
    forecast_HW = pd.DataFrame(forecast_HW, columns = ['Date', 'Time', 'DateTime', 'Air Temp', 'Sol Rad',
                                                        'Temp_Actual', 'HW', 'id_no'])
    forecast_HW['HW error_2'] = (forecast_HW['Temp_Actual'] - forecast_HW['HW'])**2
    return forecast_HW

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
forecast_HW = Holt_Winters_forecast()

In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp, h, theta_ideal):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 2*(-hive_temp + env_temp) 
    dydt = M + W 
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp, h, theta_ideal):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp, h, theta_ideal), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
    theta_t = sol[-1,:]
    return theta_t    

In [ ]:
def residual(ps, ts, data, l, argv):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    r_c = 81 #ps['w1'].value
    r_h = 220 #ps['w2'].value
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value #health factor per day
        theta_ideal = ps['theta_'+str(k)].value + 273.15 #theta_ideal per day
        t_max = len(n[argv[1]])-1
        t = np.linspace(0,t_max, num = t_max+1)
        environment_temp = n[argv[1]].values + 273.15 #hourly environment temperature -> kelvin
        
        a = [1]*len(environment_temp)
        
        fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, theta_ideal)
        model = np.concatenate((model, fitted - 273.15))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data[argv[3]]).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    return return_value

In [ ]:
def main(*argv):
    data = pd.read_csv(argv[0])
    try:
        for i in range(len(data)):
            current_date = (datetime.datetime.strptime(str(data['Date'].values[i]), "%m/%d/%Y")).date()
            current_time = datetime.datetime.strptime(str(data['Time'].values[i]), "%H:%M:%S").time()
            current_datetime = datetime.datetime.combine(current_date,current_time)
            data['DateTime'][i] = datetime.datetime.strptime(str(current_datetime), "%Y-%m-%d %H:%M:%S")
            data['Date'][i] = data['DateTime'][i].date()
            data['Time'][i] = data['DateTime'][i].time()
    except:
        pass
    d = pd.DataFrame(data).groupby([data['Date']])
    h_max = 1
    l = 96/h_max

    forecast_EBVplus = np.array([[],[],[],[],[],[],[],[]]).reshape(-1,8)
    id_s = 0
    for i in range(0,int(len(data)/daily_data_points)-input_cycle-output_cycle+1):
        print(i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points] #whole dataset -> input_cycle+output_cycle
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        exogenous_train = pd.DataFrame(d_in[0:len(data_in)]) #input exogenous data
        data_input = d_in[0:daily_data_points*input_cycle]
        data_temp_input = d_in[argv[3]][0:daily_data_points*input_cycle] #7 days hive core temperature for forecasting -> celcius
        data_ext_input = d_in[argv[1]][0:daily_data_points*input_cycle] #7 days environment temperature for forecasting -> celcius

        params = Parameters()
        for j in range(7):
            params.add('h_'+str(j), value = 0.5, min = 0.2, max = h_max)
            params.add('theta_'+str(j), value = 35, min = 33, max = 38)   
        t_max = len(data_ext_input)-1
        t = np.linspace(0,t_max, num = t_max+1)
        result = minimize(residual, params, args=(t,data_input, l, argv),
                          method='leastsq',nan_policy='omit',max_nfev = 300)
        sig_min = data_temp_input + result.residual[:len(data_temp_input)].reshape(data_temp_input.shape)
        

        param_min_h,param_min_theta = [], []
        for m in range(input_cycle):
            param_min_h.append(result.params['h_'+str(m)].value)
            param_min_theta.append(result.params['theta_'+str(m)].value)
            
        ##one day ahead forecast 
        forecast_h = param_min_h[-1]
        forecast_theta = param_min_theta[-1]

        h = forecast_h
        theta_ideal = forecast_theta + 273.15
        #theta_ideal = result.params['theta_'+str(6)].value + 273.15
        
        last_date_input = d_in[0:input_cycle*daily_data_points]['Date'].values[-1]
        forecast_data = d_in[input_cycle*daily_data_points:(input_cycle+output_cycle)*daily_data_points]
        forecast_data = forecast_data.reset_index() 
        
        for p in range(len(forecast_data)):
            forecast_data_date = datetime.datetime.strptime(str(forecast_data['Date'].values[p]),  "%Y-%m-%d").date() 
            forecast_data_time = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            forecast_data['DateTime'][p] = str(datetime.datetime.combine(forecast_data_date,forecast_data_time))
    
        #print('forecast_data',forecast_data)
        current_date = np.array([[],[],[]]).reshape(-1,3)
        for p in range(output_cycle*daily_data_points):
            current_date_forecast = (datetime.datetime.strptime(str(last_date_input), "%Y-%m-%d") + datetime.timedelta(days=int(p/daily_data_points) + 1)).date()
            current_time_forecast = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            current_datetime_forecast = datetime.datetime.combine(current_date_forecast,current_time_forecast)
            current_date = np.concatenate((current_date, np.array([str(current_date_forecast),
                                                                    str(current_time_forecast),
                                                                    str(current_datetime_forecast)]).reshape(3,-1).T))
        current_date = pd.DataFrame(current_date, columns = ['Date','Time','DateTime'])
        #print('current_data',current_date)            
        forecast_data = forecast_data.merge(current_date, how = 'outer', left_on = 'DateTime', right_on = 'DateTime')
        #print('forecast_data_merged_first',forecast_data)
        forecast_data = forecast_data[~np.isnan(forecast_data['Temp'].values)]
        forecast_data = forecast_data[~pd.isnull(forecast_data['Date_y'].values)]
        #print('forecast_data_merged',forecast_data)
        forecast_data_partial = forecast_data[['Date_y', 'Time_y', 'DateTime', 'Air Temp', 'Sol Rad', 'Temp']]
        forecast_data_partial = forecast_data_partial.rename(columns={"Date_y": "Date", "Time_y": "Time"})
        
        t = np.linspace(0, len(forecast_data_partial)-1, num = len(forecast_data_partial)) #forecast hourly
        a = [1]*len(t)
        r_c = 81 #ps['w1'].value
        r_h = 220 #ps['w2'].value
        
        env_temp_future = forecast_data_partial[argv[1]] + 273.15
        solar_rad_future = forecast_data_partial[argv[2]]
        hive_temp_future = forecast_data_partial[argv[3]]
        
        forecasted = run_bee_eq(t, a, r_c, r_h, env_temp_future, h, theta_ideal) - 273.15
        print(len(forecasted))
        id_no = [id_s]*len(forecasted)
        id_s = id_s + 1
        forecast_EBVplus = np.concatenate((forecast_EBVplus, np.array([forecast_data_partial['Date'].values, 
                                                               forecast_data_partial['Time'].values,
                                                               forecast_data_partial['DateTime'].values,  
                                                               forecast_data_partial['Air Temp'].values,         
                                                               forecast_data_partial['Sol Rad'].values,
                                                               forecast_data_partial['Temp'].values,
                                                               np.array(forecasted),
                                                               np.array(id_no)]).reshape(8,-1).T))
        
        plt.figure(figsize = (10,4), dpi = 200)
        plt.plot(hive_temp_future, label = 'Hive Core Temperature', color = 'blue')
        plt.plot(forecasted, label = 'Forecasted Core Temp', color = 'red') 
        plt.xlabel('Samples per hour')
        plt.ylabel('Temperature in celcius')
        plt.legend()
        plt.show()
    forecast_EBVplus = pd.DataFrame(forecast_EBVplus, columns = ['Date', 'Time', 'DateTime', 'Air Temp', 'Sol Rad',
                                                        'Temp_Actual', 'EBV+', 'id_no'])
    forecast_EBVplus['EBV+ error_2'] = (forecast_EBVplus['Temp_Actual'] - forecast_EBVplus['EBV+'])**2
    
    return forecast_EBVplus

In [ ]:
if __name__ == "__main__":
    filename = 'C:\Bee Temp Data\C294_w_Env.csv'
    environment_temperature = 'Air Temp'
    solar_radiation = 'Sol Rad'
    hive_temperature = 'Temp'
    forecast_EBVplus = main(filename, environment_temperature, solar_radiation, hive_temperature)

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal, k_c, P_C):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_c + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W - P_C
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, k_c, P_C):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal, k_c, P_C), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
def residual(ps, ts, data, l, argv, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, k_c, P_C):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    r_c = 124 #ps['w1'].value
    r_h = 428 #ps['w2'].value
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value #health factor per day
        theta_ideal = ps['theta_'+str(k)].value + 273.15 #theta_ideal per day
        t_max = len(n[argv[1]])-1
        t = np.linspace(0,t_max, num = t_max+1)
        environment_temp = n[argv[1]].values + 273.15 #hourly environment temperature -> kelvin
        solar_rad = n[argv[2]].values #hourly solar radiation -> kelvin
        P_C = n['P'].values
        
        a = [1]*len(environment_temp)
        
        fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                            k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, k_c, P_C)
        model = np.concatenate((model, fitted - 273.15))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data[argv[3]]).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    #print(np.mean(abs((model - data[argv[3]]).ravel())))
    return return_value

In [ ]:
def main(*argv):
    data = pd.read_csv(argv[0])
    try:
        for i in range(len(data)):
            current_date = (datetime.datetime.strptime(str(data['Date'].values[i]), "%m/%d/%Y")).date()
            current_time = datetime.datetime.strptime(str(data['Time'].values[i]), "%H:%M:%S").time()
            current_datetime = datetime.datetime.combine(current_date,current_time)
            data['DateTime'][i] = datetime.datetime.strptime(str(current_datetime), "%Y-%m-%d %H:%M:%S")
            data['Date'][i] = data['DateTime'][i].date()
            data['Time'][i] = data['DateTime'][i].time()
    except:
        pass
    d = pd.DataFrame(data).groupby([data['Date']])
    k_W = 0.29 #thermal conductivity of wood -> per hour
    eta_W = 0.0085*k_W #heat absorption coeff of wood
    k_A_W = k_W #thermal conductivity of top surface (wood + aluminum plate) -> per hour
    eta_A_W = eta_W #heat absorption coeff of top surface
    eta_B_W = eta_W #heat absorption coeff of bottom surface
    k_hive = 0.8
    k_c = 0
    h_max = 1
    l = 96/h_max

    forecast_ABK = np.array([[],[],[],[],[],[],[],[]]).reshape(-1,8)
    id_s = 0
    for i in range(0,int(len(data)/daily_data_points)-input_cycle-output_cycle+1):
        print(i)
        d_in =  data[i*daily_data_points:(i+input_cycle+output_cycle)*daily_data_points] #whole dataset -> input_cycle+output_cycle
        data_in = d_in[0:input_cycle*daily_data_points]['Temp'] #input core temperature data
        exogenous_train = pd.DataFrame(d_in[0:len(data_in)]) #input exogenous data
        data_input = d_in[0:daily_data_points*input_cycle]
        data_temp_input = d_in[argv[3]][0:daily_data_points*input_cycle] #7 days hive core temperature for forecasting -> celcius
        data_ext_input = d_in[argv[1]][0:daily_data_points*input_cycle] #7 days environment temperature for forecasting -> celcius
        P_C = data_input['P'][0:24*7]
        params = Parameters()
        for j in range(7):
            params.add('h_'+str(j), value = 0.5, min = 0.2, max = h_max)
            params.add('theta_'+str(j), value = 35, min = 33, max = 38)   
        t_max = len(data_ext_input)-1
        t = np.linspace(0,t_max, num = t_max+1)
        result = minimize(residual, params, args=(t,data_input, l, argv, k_W, k_A_W, 
                                                  k_hive, eta_W, eta_A_W, eta_B_W, k_c, P_C),
                          method='leastsq',nan_policy='omit',max_nfev = 300)
        sig_min = data_temp_input + result.residual[:len(data_temp_input)].reshape(data_temp_input.shape)
        

        param_min_h,param_min_theta = [], []
        for m in range(input_cycle):
            param_min_h.append(result.params['h_'+str(m)].value)
            param_min_theta.append(result.params['theta_'+str(m)].value)
            
        ##one day ahead forecast 
        forecast_h = param_min_h[-1]
        forecast_theta = param_min_theta[-1]

        h = forecast_h
        theta_ideal = forecast_theta + 273.15
        #theta_ideal = result.params['theta_'+str(6)].value + 273.15
        
        last_date_input = d_in[0:input_cycle*daily_data_points]['Date'].values[-1]
        forecast_data = d_in[input_cycle*daily_data_points:(input_cycle+output_cycle)*daily_data_points]
        forecast_data = forecast_data.reset_index() 
        
        for p in range(len(forecast_data)):
            forecast_data_date = datetime.datetime.strptime(str(forecast_data['Date'].values[p]),  "%Y-%m-%d").date() 
            forecast_data_time = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            forecast_data['DateTime'][p] = str(datetime.datetime.combine(forecast_data_date,forecast_data_time))
    
        #print('forecast_data',forecast_data)
        current_date = np.array([[],[],[]]).reshape(-1,3)
        for p in range(output_cycle*daily_data_points):
            current_date_forecast = (datetime.datetime.strptime(str(last_date_input), "%Y-%m-%d") + datetime.timedelta(days=int(p/daily_data_points) + 1)).date()
            current_time_forecast = datetime.datetime.strptime(str(forecast_data['Time'].values[p]), "%H:%M:%S").time()
            current_datetime_forecast = datetime.datetime.combine(current_date_forecast,current_time_forecast)
            current_date = np.concatenate((current_date, np.array([str(current_date_forecast),
                                                                    str(current_time_forecast),
                                                                    str(current_datetime_forecast)]).reshape(3,-1).T))
        current_date = pd.DataFrame(current_date, columns = ['Date','Time','DateTime'])
        #print('current_data',current_date)            
        forecast_data = forecast_data.merge(current_date, how = 'outer', left_on = 'DateTime', right_on = 'DateTime')
        #print('forecast_data_merged_first',forecast_data)
        forecast_data = forecast_data[~np.isnan(forecast_data['Temp'].values)]
        forecast_data = forecast_data[~pd.isnull(forecast_data['Date_y'].values)]
        #print('forecast_data_merged',forecast_data)
        forecast_data_partial = forecast_data[['Date_y', 'Time_y', 'DateTime', 'Air Temp', 'Sol Rad', 'Temp']]
        forecast_data_partial = forecast_data_partial.rename(columns={"Date_y": "Date", "Time_y": "Time"})
        
        t = np.linspace(0, len(forecast_data_partial)-1, num = len(forecast_data_partial)) #forecast hourly
        a = [1]*len(t)
        r_c = 124#ps['w1'].value
        r_h = 428 #['w2'].value
        
        env_temp_future = forecast_data_partial[argv[1]] + 273.15
        solar_rad_future = forecast_data_partial[argv[2]]
        hive_temp_future = forecast_data_partial[argv[3]]
        
        forecasted = run_bee_eq(t, a, r_c, r_h, env_temp_future, h, 
                                         solar_rad_future, 
                                         k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, 0, 0) - 273.15
        id_no = [id_s]*len(forecasted)
        id_s = id_s + 1
        forecast_ABK = np.concatenate((forecast_ABK np.array([forecast_data_partial['Date'].values, 
                                                               forecast_data_partial['Time'].values,
                                                               forecast_data_partial['DateTime'].values,  
                                                               forecast_data_partial['Air Temp'].values,         
                                                               forecast_data_partial['Sol Rad'].values,
                                                               forecast_data_partial['Temp'].values,
                                                               np.array(forecasted),
                                                               np.array(id_no)]).reshape(8,-1).T))
        
        plt.figure(figsize = (10,4), dpi = 200)
        plt.plot(hive_temp_future, label = 'Hive Core Temperature', color = 'blue')
        plt.plot(forecasted, label = 'Forecasted Core Temp', color = 'red') 
        plt.xlabel('Samples per hour')
        plt.ylabel('Temperature in celcius')
        plt.legend()
        plt.show()
    forecast_ABK = pd.DataFrame(forecast_ABK, columns = ['Date', 'Time', 'DateTime', 'Air Temp', 'Sol Rad',
                                                        'Temp_Actual', 'ABK', 'id_no'])
    forecast_ABK['ABK error_2'] = (forecast_ABK['Temp_Actual'] - forecast_ABK['ABK'])**2
    
    return forecast_ABK

In [ ]:
if __name__ == "__main__":
    filename = 'C:\Bee Temp Data\C294_w_Env.csv'
    environment_temperature = 'Air Temp'
    solar_radiation = 'Sol Rad'
    hive_temperature = 'Temp'
    forecast_EBVC = main(filename, environment_temperature, solar_radiation, hive_temperature)

In [ ]:
all_forecast = forecast_ARX.merge(forecast_sARX, how='outer', left_on = ['id_no','DateTime'], right_on = ['id_no','DateTime'])
all_forecast = all_forecast.merge(forecast_HW, how = 'outer', left_on = ['id_no','DateTime'], right_on = ['id_no','DateTime'])
all_forecast = all_forecast[['Date_x','Time_x','DateTime','id_no','Air Temp_x','Sol Rad_x','Temp_Actual_x','ARX',
                              'ARX error_2','sARX', 'sARX error_2','HW', 'HW error_2']]
all_forecast = all_forecast.merge(forecast_EBVplus, how = 'outer', left_on = ['id_no','DateTime'], right_on = ['id_no','DateTime'])
all_forecast = all_forecast[['Date_x','Time_x','DateTime','id_no','Air Temp_x','Sol Rad_x','Temp_Actual_x','ARX',
                              'ARX error_2','sARX', 'sARX error_2','HW', 'HW error_2', 'EBV+', 'EBV+ error_2']]
all_forecast = all_forecast.merge(forecast_ABK, how = 'outer', left_on = ['id_no','DateTime'], right_on = ['id_no','DateTime'])
forecasted_all = all_forecast[['Date_x','Time_x','DateTime','id_no','Air Temp_x','Sol Rad_x','Temp_Actual_x','ARX',
                              'ARX error_2','sARX', 'sARX error_2','HW', 'HW error_2', 'EBV+', 'EBV+ error_2',
                               'ABK', 'ABK error_2', ]]
forecasted_all = forecasted_all.rename(columns={"Date_x": "Date", "Time_x": "Time", "Air Temp_x": "Air Temp",
                                               "Sol Rad_x":"Sol Rad", "Temp_Actual_x":"Temp_Actual"})
forecasted_all.to_csv('C:\Bee Temp Data\ABK_Baseline_Forecast_294.csv')